In [1]:
import numpy as np

In [2]:
def random_initialization(A,rank):
    number_of_documents = A.shape[0]
    number_of_terms = A.shape[1]
    W = np.random.uniform(1,2,(number_of_documents,rank))
    H = np.random.uniform(1,2,(rank,number_of_terms))
    return W,H
                          

def nndsvd_initialization(A,rank):

    u,s,v=np.linalg.svd(A,full_matrices=False)

    v=v.T
    w=np.zeros((A.shape[0],rank))
    h=np.zeros((rank,A.shape[1]))

    w[:,0]=np.sqrt(s[0])*np.abs(u[:,0])
    h[0,:]=np.sqrt(s[0])*np.abs(v[:,0].T)

    for i in range(1,rank):
        
        ui=u[:,i]
        vi=v[:,i]
        ui_pos=(ui>=0)*ui
        ui_neg=(ui<0)*-ui
        vi_pos=(vi>=0)*vi
        vi_neg=(vi<0)*-vi
        
        ui_pos_norm=np.linalg.norm(ui_pos,2)
        ui_neg_norm=np.linalg.norm(ui_neg,2)
        vi_pos_norm=np.linalg.norm(vi_pos,2)
        vi_neg_norm=np.linalg.norm(vi_neg,2)
        
        norm_pos=ui_pos_norm*vi_pos_norm
        norm_neg=ui_neg_norm*vi_neg_norm
        
        if norm_pos>=norm_neg:
            w[:,i]=np.sqrt(s[i]*norm_pos)/ui_pos_norm*ui_pos
            h[i,:]=np.sqrt(s[i]*norm_pos)/vi_pos_norm*vi_pos.T
        else:
            w[:,i]=np.sqrt(s[i]*norm_neg)/ui_neg_norm*ui_neg
            h[i,:]=np.sqrt(s[i]*norm_neg)/vi_neg_norm*vi_neg.T

    return w,h

In [ ]:
# def mur_optimization_RNMF( X, k, max_iteration, init_mode):

#     #X_c = the X with clean data
#     #X is the polluted data


#     #U.shape is m * k,  V.shape is k * n
#     #m = numbers of features/ elements of 1 column vector of X.
#     #n is the numbers of training examples

#     #TODO

#     #implement earlier stop


#     #X should be 

#     if init_mode == 'random':
#         U ,V = random_initialization(X_c,k)    
#     elif init_mode == 'nndsvd':
#         U ,V = nndsvd_initialization(X_c,k) 

    
#     error = []

#     e = 1.0e-10

#     for iter in range(max_iteration):

#         #update U






    

In [3]:
def mur_optimization_nmf_stock(X, k, max_iter, init_mode):

    if init_mode == 'random':
        U ,V = random_initialization(X, k)
    elif init_mode == 'nndsvd':
        U ,V = nndsvd_initialization(X, k) 


    loss_iter = []


    for iter in range(max_iter):

        #update V
        U_tX = U.T @ X
        U_tUV = U.T @ U @ V

        for i in range(np.size(V, 0)): #i row
            for j in range(np.size(V, 1)): #j column
                V[i,j] = V[i,j] * U_tX[i,j] / U_tUV[i,j]


        #update U
        XV_t = X @ V.T
        UVV_t = U @ V @ V.T

        for i in range(np.size(U, 0)): #i row
            for j in range(np.size(U, 1)): #j column
                U[i,j] = U[i,j] * XV_t[i,j] / UVV_t[i,j]

        
        loss = np.linalg.norm(X - (U @ V), 'fro' )  #Frobenius norm, by default
        loss_iter.append(loss)
    

    return U, V, loss_iter




In [ ]:
def mur_optimization_robust_nmf_l1(X, k, max_iter, init_mode):
    
    if init_mode == 'random':
        U ,V = random_initialization(X, k)
    elif init_mode == 'nndsvd':
        U ,V = nndsvd_initialization(X, k) 


    #generate E, E is initialized with all zeros
    #inefficient, but conservative
    E = np.zeros(X.shape)



    #E = np.random.random(())


    
    loss_iter = []

    
    for iter in range(max_iter):


        # update U

        #get X^
        X_hat = np.subtract(X,E)

        #check negative constrain  X - E >= 0
        if X_hat.sum() <= 0:
            print("Error, noise is greater than X")


        X_hat_V_t = X_hat @ V.T
        UVV_t = U @ V @ V.T

        for i in range(np.size(U, 0)): #i row
            for j in range(np.size(U,1)): #j column
                U[i,j] = U[i,j] * X_hat_V_t[i,j] / UVV_t[i,j]
        

        # update V, E simultaneously

        E_p = E
        E_n = np.zeros(X.shape)

        #compute V dash
        V_dash = np.concatenate((V, E_p, E_n), axis=0)

        #compute X dash
        X_dash = np.concatenate( ( X, np.zeros((1, X.shape[1])) ), axis=0)


        #compute U dash
        
        




        






In [18]:
X = np.random.random((3,3))
print(X, X.sum(), X.shape)

Y = np.random.random(X.shape)

r = np.subtract(X, Y)

print(Y, Y.sum(), Y.shape)

print(r, r.sum(), r.shape)

c = np.concatenate((X,Y,r, np.zeros((1, X.shape[1]))), axis=0)

print(c, c.shape)
c_n = np.negative(r)
print(c_n, c_n.shape)

[[0.90113799 0.96800404 0.14544747]
 [0.70992343 0.322142   0.28333487]
 [0.93181975 0.02863305 0.15331578]] 4.443758375968689 (3, 3)
[[0.18720333 0.51781043 0.30718818]
 [0.34535952 0.50280706 0.11993728]
 [0.37608072 0.70091447 0.33370107]] 3.391002080128805 (3, 3)
[[ 0.71393466  0.45019361 -0.16174072]
 [ 0.3645639  -0.18066506  0.16339759]
 [ 0.55573903 -0.67228142 -0.18038529]] 1.0527562958398842 (3, 3)
[[ 0.90113799  0.96800404  0.14544747]
 [ 0.70992343  0.322142    0.28333487]
 [ 0.93181975  0.02863305  0.15331578]
 [ 0.18720333  0.51781043  0.30718818]
 [ 0.34535952  0.50280706  0.11993728]
 [ 0.37608072  0.70091447  0.33370107]
 [ 0.71393466  0.45019361 -0.16174072]
 [ 0.3645639  -0.18066506  0.16339759]
 [ 0.55573903 -0.67228142 -0.18038529]
 [ 0.          0.          0.        ]] (10, 3)
[[-0.71393466 -0.45019361  0.16174072]
 [-0.3645639   0.18066506 -0.16339759]
 [-0.55573903  0.67228142  0.18038529]] (3, 3)
